In [1]:
import pandas as pd
import numpy as np

In [2]:
Questions = pd.read_csv("Preprocessed_questions.csv")

C:\Users\beboz\.conda\envs\computervision\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Questions.shape

(1033747, 7)

In [4]:
Questions.head()

,Unnamed: 0,Id,OwnerUserId,CreationDate,Score,Title,Body
0,0,80,26,2008-08-01T13:57:07Z,26,SQLStatement.execute() - multiple queries in o...,i 've write database generation script in sql ...
1,1,90,58,2008-08-01T14:41:24Z,144,Good branching and merging tutorials for Torto...,be there really good tutorial explain branchin...
2,2,120,83,2008-08-01T15:50:08Z,21,ASP.NET Site Maps,have anyone get experience create sql-based as...
3,3,180,2.08974e+06,2008-08-01T18:42:19Z,53,Function for creating color wheels,be something i 've pseudo-solved many time nev...
4,4,260,91,2008-08-01T23:22:08Z,49,Adding scripting functionality to .NET applica...,i have little game write in c use database as ...


In [5]:
Questions.dtypes

Unnamed: 0       int64
Id              object
OwnerUserId     object
CreationDate    object
Score           object
Title           object
Body            object
dtype: object

In [6]:
def intify(string):
    try:
        return int(string)
    except:
        return None

In [7]:
Questions["Score"] = Questions["Score"].apply(lambda x: intify(x))

In [9]:
Questions.drop(Questions[Questions["Score"] < 5].index, inplace = True)

In [8]:
Questions.dropna(inplace = True)

In [10]:
np.median(Questions["Score"])

8.0

In [11]:
Questions.shape

(85919, 7)

In [12]:
Tags = pd.read_csv("Filtered_tags.csv")

In [13]:
tag_map = {
    "html5":"html",
    "iphone":"ios",
    "ruby-on-rails":"ruby",
    "ruby-on-rails-3":"ruby",
    "asp.net-mvc":"asp.net",
    "xcode":"ios",
    "vba":"excel",
    "python-2.7":"python",
    "css3":"css",
    "excel-vba":"excel",
    "sql-server-2008":"sql-server",
    "visual-studio-2010":"visual-studio",
    "google-maps":"google",
    "google-chrome":"google",
    "jsp":"java",
    "python-3.x":"python"
}

In [14]:
def reset_tags(tag):
    try:
        return tag_map[tag]
    except:
        return tag

In [15]:
Tags["Tag"] = Tags["Tag"].apply(lambda x : reset_tags(x))

In [16]:
Tags.shape

(1755529, 3)

In [17]:
Tags = Tags[["Id","Tag"]]
Tags = Tags.drop_duplicates()

In [18]:
ids = list(set(Tags["Id"]) & set(Questions["Id"]))

In [19]:
len(ids)

34533

In [20]:
Tags.drop(Tags[~Tags["Id"].isin(ids)].index, inplace = True)

In [21]:
Tags.shape

(51617, 2)

In [22]:
tags_to_remove = Tags["Tag"].value_counts()[Tags["Tag"].value_counts() < 300].index

In [23]:
Tags.drop(Tags[Tags["Tag"].isin(tags_to_remove)].index, inplace = True)

In [24]:
Tags = Tags.groupby("Id",as_index=False).agg(lambda x: ','.join(x))
Tags["Tag"] = Tags["Tag"].apply(lambda x:x.split(','))

In [25]:
data = pd.merge(Questions,Tags,on="Id")

In [26]:
data.shape

(32403, 8)

In [27]:
data = data[["Title","Body","Tag"]]

In [59]:
Body = data[["Title","Body"]]
Tag = data["Tag"]

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support

In [30]:
binarizer = MultiLabelBinarizer()

In [60]:
Tag = binarizer.fit_transform(Tag)

In [32]:
len(binarizer.classes_)

40

In [33]:
Tag = pd.DataFrame(Tag,columns = binarizer.classes_)

In [34]:
data.head()

,Title,Body,Tag
0,ASP.NET Site Maps,have anyone get experience create sql-based as...,"[sql, asp.net]"
1,Function for creating color wheels,be something i 've pseudo-solved many time nev...,[algorithm]
2,Adding scripting functionality to .NET applica...,i have little game write in c use database as ...,"[c#, .net]"
3,Should I use nested classes in this case?,i be work on collection of class use for video...,[c++]
4,Homegrown consumption of web services,i 've be write few web service for .net app no...,[.net]


In [120]:
tfidf = TfidfVectorizer(max_features = 2000,ngram_range=(1,2), stop_words = 'english')

In [121]:
Body_features = tfidf.fit_transform(Body["Body"])

In [122]:
Title_features = tfidf.fit_transform(Body["Title"])

In [69]:
Body_features = pd.DataFrame(Body_features.toarray())
Title_features = pd.DataFrame(Title_features.toarray())
features = pd.concat([Title_features,Body_features], axis=1)

In [147]:
X_train,X_test,Y_train,Y_test = train_test_split(features, Tag,test_size = 0.2,random_state = 0)
X_train,X_val,Y_train,Y_val = train_test_split(X_train, Y_train,test_size = 0.2,random_state = 0)

In [71]:
clf = OneVsRestClassifier(LinearSVC())

In [72]:
clf.fit(X_train,Y_train)

OneVsRestClassifier(estimator=LinearSVC())

In [73]:
Y_pred = clf.predict(X_test)

In [74]:
score = np.minimum(Y_test,Y_pred).sum(axis = 1) / np.maximum(Y_test,Y_pred).sum(axis = 1)

In [75]:
print("Score: "+ str(score.mean()*100))

Score: 59.322378233811655


In [123]:
import joblib
joblib.dump(clf,filename = "SVMClassifier.sav")
joblib.dump(binarizer,filename = "Binarizer.sav")
joblib.dump(tfidf,filename = "TFIDF.sav")

['TFIDF.sav']

In [100]:
title = "saved tables not showing up in object explorer in java"
body = "public static void main(string[] args)"

tt = tfidf.transform([title])
bt = tfidf.transform([body])

In [101]:
tt = tt.toarray()

In [102]:
bt = bt.toarray()

In [103]:
tt = tt.tolist()

In [104]:
bt = bt.tolist()

In [105]:
tt = tt[0]

In [106]:
bt = bt[0]

In [107]:
tt.extend(bt)

In [108]:
len(tt)

4000

In [112]:
X_test.iloc[0]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1995    0.0
1996    0.0
1997    0.0
1998    0.0
1999    0.0
Name: 15867, Length: 4000, dtype: float64

In [114]:
clf.predict([X_test.iloc[0]])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [124]:
import joblib
clf = joblib.load("SVMClassifier.sav")
binarizer = joblib.load("Binarizer.sav")
tfidf = joblib.load("TFIDF.sav")

def classify(title, body):
    tt = tfidf.transform([title])
    bt = tfidf.transform([body])
    tt = tt.toarray().tolist()
    bt = bt.toarray().tolist()
    tt = tt[0]
    tt.extend(bt[0])
    res = clf.predict([tt])
    return binarizer.inverse_transform(res)[0]

In [140]:
classify("Adding entry to unordered map ", "how to add entries to unordered map, i have key name value \"Eyad\" i want to add it as a new entry in my javascript c++ c++ c++ unordered map i tried map.insert(name,\"Eyad\")")

[('java',)]

In [155]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = 4000, output_dim=40, max_length = 4000 ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(80, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(40, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [156]:
import tensorflow as tf
model_0 = define_model( input_dim=4000, max_length=4000)
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)
model = define_model(3, activation='relu', input_dim=4000, max_length=4000)
model.fit(X_train, Y_train, epochs=10, verbose=1, callbacks=[callbacks], validation_data=(X_val, Y_val))

Epoch 1/10
649/649 [==============================] - 3933s 6s/step - loss: nan - accuracy: 0.0638 - val_loss: nan - val_accuracy: 0.0608
Epoch 2/10
649/649 [==============================] - 3767s 6s/step - loss: nan - accuracy: 0.0665 - val_loss: nan - val_accuracy: 0.0608
Epoch 3/10
 19/649 [..............................] - ETA: 55:15 - loss: nan - accuracy: 0.0658

KeyboardInterrupt: 

In [ ]:
# print("Compile model ...")
# estimator = KerasClassifier(build_fn=build_model, epochs=15, batch_size=128)
# dummy_y_train = np_utils.to_categorical(train_y)
# estimator.fit(xtrain_tfidf, dummy_y_train)

In [ ]:
# predictions = estimator.predict(xvalid_tfidf)
# y_pred = encoder.inverse_transform(predictions)
# print("accuracy=",metrics.accuracy_score(predictions, valid_y))
# conf_matrix(valid_y, predictions, df["label"])